# 02 — Data Quality Quick Viz
Scans Parquet files under `data/prices_1d` (or `prices_1h`) and shows freshness, coverage, NaNs, duplicates, weekend bars, and ADR stats.

In [ ]:

from pathlib import Path
import sys

PROJECT_OVERRIDE = r"C:\Users\speed\Desktop\Forex CFD's system"
ROOT = Path(PROJECT_OVERRIDE)
DATA_1D = ROOT / "data" / "prices_1d"
DATA_1H = ROOT / "data" / "prices_1h"
CFG = ROOT / "config" / "baseline.yaml"
DATA_DIR = DATA_1D if DATA_1D.exists() else DATA_1H

print("Project root:", ROOT)
print("Using data dir:", DATA_DIR)
assert DATA_DIR.exists(), f"Expected data under {ROOT/'data'} (prices_1d or prices_1h)"
assert CFG.exists(), f"Missing baseline config at {CFG}"


In [ ]:
import pandas as pd, numpy as np
from pathlib import Path
files = sorted(Path(DATA_DIR).glob('*.parquet'))
assert files, f'No parquet files in {DATA_DIR}'
rows = []
for p in files:
    df = pd.read_parquet(p)
    if not isinstance(df.index, pd.DatetimeIndex):
        df = df.set_index(pd.to_datetime(df['Date'], utc=True, errors='coerce'))
    if df.index.tz is None:
        df.index = df.index.tz_localize('UTC')
    df = df.sort_index()
    dup_ts = int(df.index.duplicated().sum())
    weekends = df.index.weekday >= 5
    weekend_pct = float(100*weekends.mean())
    nans = {c:int(df[c].isna().sum()) for c in ['Open','High','Low','Close','Volume'] if c in df.columns}
    adr = (df['High']-df['Low']).abs().resample('D').mean().median() if 'High' in df and 'Low' in df else np.nan
    rows.append({
        'symbol': p.stem.upper(), 'tf': DATA_DIR.name.split('_')[-1],
        'bars': len(df), 'start_utc': str(df.index.min()), 'end_utc': str(df.index.max()),
        'dup_ts': dup_ts, 'weekend_pct': weekend_pct, 'adr_median': adr,
        **{f'nan_{k}':v for k,v in nans.items()}
    })
table = pd.DataFrame(rows).sort_values('symbol').reset_index(drop=True)
from caas_jupyter_tools import display_dataframe_to_user
display_dataframe_to_user('Data quality summary', table)
table.head()

In [ ]:
import plotly.express as px
table['end_dt'] = pd.to_datetime(table['end_utc'])
fig = px.bar(table, x='symbol', y='bars', title='Bars per symbol')
fig.show()
fig2 = px.bar(table, x='symbol', y='weekend_pct', title='Weekend bars (% of rows)')
fig2.show()
fig3 = px.bar(table, x='symbol', y='adr_median', title='Median ADR (abs(High-Low))')
fig3.show()
